In [1]:
import os 
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig, pipeline, logging
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")

In [2]:
dataset="bentrevett/multi30k"
model_id="microsoft/phi-2"
output_model="phi2-multi30k-v1"

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
# Instruct chat template chat template
#inputs = tokenizer("Instruct: translate the guy with the orange hat is staring at something to german\nOutput:", return_tensors="pt", return_attention_mask=False)

inputs = tokenizer("Instruct: translate English to German.\nInput: the guy with the orange hat is staring at something.\nOutput:", return_tensors="pt", return_attention_mask=False)
outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct: translate English to German.
Input: the guy with the orange hat is staring at something.
Output: Der Mann mit der orange Haut ist ganz zurückgehend.
<|endoftext|>


In [8]:
from transformers import GenerationConfig
from time import perf_counter

def formatted_prompt(question)-> str:
    return f"Instruct: Translate German to English.\nInput: {question}\nOutput:"

def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=20,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  output=tokenizer.decode(outputs[0], skip_special_tokens=True)
  #print(output)
  output_time = perf_counter() - start_time
  #print(f"Time taken for inference: {round(output_time,2)} seconds")
  return output

In [5]:
data_test = load_dataset(dataset, split="test")

In [9]:
import re
mydf = pd.DataFrame(columns=['de','en','predicted_de'])
for i in tqdm(range(0,len(data_test))):
    out=generate_response(user_input=data_test[i]['de'])
    out=out.split('Output: ')[1]
    #processed_out=re.sub("[^A-Za-z0-9., ]","",out)
    mydf=pd.concat([mydf,pd.DataFrame({'de':data_test[i]['de'],'en':data_test[i]['en'],'predicted_en':out},index=[0])],ignore_index=True)
    

100%|██████████| 1000/1000 [09:26<00:00,  1.77it/s]


In [10]:
mydf.to_csv('phi2-zeroshot-multi30k-v1-GER-EN.csv',index=False)